# About this notebook

This notebook is for test running the data pipeline and  as a sandbox for testing new functions that we are adding to data pipeline.

In [1]:
# import packages
import numpy as np
import pandas as pd
import argparse
import os

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../hourly-egrid/")

# import local modules
import src.data_cleaning as data_cleaning
import src.load_data as load_data
import src.impute_hourly_profiles as impute_hourly_profiles
import src.eia930 as eia930
import src.output_data as output_data

from src.column_checks import get_dtypes, apply_dtypes

year = 2020

# Run the Pipeline

In [ ]:
%cd ../src
%run data_pipeline --year 2020

In [ ]:
%cd ../src
%run data_pipeline --small SMALL --year 2020

# Functions for loading intermediate outputs

In [ ]:

# load data from csv
year = 2020
path_prefix = ''

cems = pd.read_csv(f'../data/outputs/{path_prefix}{year}/cems_{year}.csv', dtype=get_dtypes(), parse_dates=['datetime_utc', 'report_date'])
partial_cems_scaled = pd.read_csv(f'../data/outputs/{path_prefix}{year}/partial_cems_scaled_{year}.csv', dtype=get_dtypes(), parse_dates=['datetime_utc', 'report_date'])
eia923_allocated = pd.read_csv(f'../data/outputs/{path_prefix}{year}/eia923_allocated_{year}.csv', dtype=get_dtypes(), parse_dates=['report_date'])
plant_attributes = pd.read_csv(f"../data/outputs/{path_prefix}{year}/plant_static_attributes_{year}.csv")
primary_fuel_table = plant_attributes.drop_duplicates(subset="plant_id_eia")[["plant_id_eia", "plant_primary_fuel"]]
residual_profiles = pd.read_csv(f"../data/outputs/{path_prefix}{year}/residual_profiles_{year}.csv")
shaped_eia_data = pd.read_csv(f"../data/outputs/{path_prefix}{year}/shaped_eia923_data_{year}.csv")

In [2]:
# load data from csv
year = 2020
path_prefix = ''

shaped_eia_data = pd.read_csv(f"../data/outputs/{path_prefix}{year}/shaped_eia923_data_{year}.csv")

In [4]:
shaped_eia_data[shaped_eia_data["ba_code"] == "CISO"]

,plant_id_eia,ba_code,fuel_category,datetime_utc,report_date,profile_method,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,net_generation_mwh,co2_mass_lb,ch4_mass_lb,n2o_mass_lb,co2e_mass_lb,nox_mass_lb,so2_mass_lb,co2_mass_lb_for_electricity,ch4_mass_lb_for_electricity,n2o_mass_lb_for_electricity,co2e_mass_lb_for_electricity,nox_mass_lb_for_electricity,so2_mass_lb_for_electricity,co2_mass_lb_adjusted,ch4_mass_lb_adjusted,n2o_mass_lb_adjusted,co2e_mass_lb_adjusted,nox_mass_lb_adjusted,so2_mass_lb_adjusted,co2_mass_lb_for_electricity_adjusted,ch4_mass_lb_for_electricity_adjusted,n2o_mass_lb_for_electricity_adjusted,co2e_mass_lb_for_electricity_adjusted,nox_mass_lb_for_electricity_adjusted,so2_mass_lb_for_electricity_adjusted
367391,903901,CISO,biomass,2020-01-01 08:00:00+00:00,2020-01-01,assumed_flat,8844.813844,7790.624194,572.565726,1.599878e+06,416.499194,54.439113,1.630262e+06,4284.029839,235.003091,1.397648e+06,355.247581,46.414113,1.423558e+06,3670.711828,211.513978,18039.407258,411.184946,53.913978,48086.155914,4108.140995,149.961694,12932.79328,349.986022,45.894355,38508.789382,3496.575672,127.483871
367392,903901,CISO,biomass,2020-01-01 09:00:00+00:00,2020-01-01,assumed_flat,8844.813844,7790.624194,572.565726,1.599878e+06,416.499194,54.439113,1.630262e+06,4284.029839,235.003091,1.397648e+06,355.247581,46.414113,1.423558e+06,3670.711828,211.513978,18039.407258,411.184946,53.913978,48086.155914,4108.140995,149.961694,12932.79328,349.986022,45.894355,38508.789382,3496.575672,127.483871
367393,903901,CISO,biomass,2020-01-01 10:00:00+00:00,2020-01-01,assumed_flat,8844.813844,7790.624194,572.565726,1.599878e+06,416.499194,54.439113,1.630262e+06,4284.029839,235.003091,1.397648e+06,355.247581,46.414113,1.423558e+06,3670.711828,211.513978,18039.407258,411.184946,53.913978,48086.155914,4108.140995,149.961694,12932.79328,349.986022,45.894355,38508.789382,3496.575672,127.483871
367394,903901,CISO,biomass,2020-01-01 11:00:00+00:00,2020-01-01,assumed_flat,8844.813844,7790.624194,572.565726,1.599878e+06,416.499194,54.439113,1.630262e+06,4284.029839,235.003091,1.397648e+06,355.247581,46.414113,1.423558e+06,3670.711828,211.513978,18039.407258,411.184946,53.913978,48086.155914,4108.140995,149.961694,12932.79328,349.986022,45.894355,38508.789382,3496.575672,127.483871
367395,903901,CISO,biomass,2020-01-01 12:00:00+00:00,2020-01-01,assumed_flat,8844.813844,7790.624194,572.565726,1.599878e+06,416.499194,54.439113,1.630262e+06,4284.029839,235.003091,1.397648e+06,355.247581,46.414113,1.423558e+06,3670.711828,211.513978,18039.407258,411.184946,53.913978,48086.155914,4108.140995,149.961694,12932.79328,349.986022,45.894355,38508.789382,3496.575672,127.483871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464010,903912,CISO,wind,2021-01-01 03:00:00+00:00,2020-12-01,residual_profile,6027.676726,6027.676726,687.071312,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
464011,903912,CISO,wind,2021-01-01 04:00:00+00:00,2020-12-01,residual_profile,5790.026389,5790.026389,659.982479,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
464012,903912,CISO,wind,2021-01-01 05:00:00+00:00,2020-12-01,residual_profile,6243.722487,6243.722487,711.697524,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
464013,903912,CISO,wind,2021-01-01 06:00:00+00:00,2020-12-01,residual_profile,5955.661472,5955.661472,678.862575,0.000000e+00,0.000000